In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


# -------------------------------------------
# Utilidades
# -------------------------------------------
def load_hourly_ws(csv_path_or_str):
    """
    Lê CSV com colunas: timestamp(YYYYMMDDHH), ws2m.
    Retorna DataFrame com índice datetime e frequência horária contínua.
    """
    if isinstance(csv_path_or_str, str) and "\n" in csv_path_or_str:
        import io
        df = pd.read_csv(io.StringIO(csv_path_or_str))
    else:
        df = pd.read_csv(csv_path_or_str)

    df["timestamp"] = pd.to_datetime(df["timestamp"].astype(str), format="%Y%m%d%H")
    df = df.set_index("timestamp").sort_index()

    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    # Grade horária contínua + preenchimento (se houver buracos)
    full_range = pd.date_range(df.index.min(), df.index.max(), freq="H")
    df = df.reindex(full_range)
    df.rename_axis("timestamp", inplace=True)
    df["ws2m"] = df["ws2m"].interpolate(method="time").ffill().bfill()

    return df[["ws2m"]].astype(float)


def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(((y_true - y_pred) ** 2).mean()))


# -------------------------------------------
# Random Forest: criação de features
# -------------------------------------------
def make_supervised(y_ser, lags_short=24, weekly_lags=(168, 167, 169), roll_windows=(3, 6, 12, 24)):
    """
    Constrói X,y (um passo à frente) com:
      - lags 1..24
      - lags semanais (se existirem dados suficientes)
      - médias e desvios móveis (3,6,12,24)
      - sen/cos da hora do dia e dia da semana
    """
    weekly_lags = tuple(L for L in weekly_lags if L < len(y_ser))
    start = max([lags_short] + list(weekly_lags) + list(roll_windows)) + 1
    if start >= len(y_ser):
        return None, None

    rows, ys = [], []
    for t in range(start, len(y_ser)):
        feats = {f"lag_{L}": y_ser.iloc[t - L] for L in range(1, lags_short + 1)}
        for L in weekly_lags:
            feats[f"lag_{L}"] = y_ser.iloc[t - L]
        for w in roll_windows:
            w_eff = min(w, t)
            feats[f"roll_mean_{w}"] = y_ser.iloc[t - w_eff:t].mean()
            feats[f"roll_std_{w}"] = y_ser.iloc[t - w_eff:t].std(ddof=0)

        ts = y_ser.index[t]
        feats["sin_h"] = np.sin(2 * np.pi * ts.hour / 24)
        feats["cos_h"] = np.cos(2 * np.pi * ts.hour / 24)
        feats["sin_w"] = np.sin(2 * np.pi * ts.dayofweek / 7)
        feats["cos_w"] = np.cos(2 * np.pi * ts.dayofweek / 7)

        rows.append(pd.Series(feats, name=ts))
        ys.append(y_ser.iloc[t])

    X = pd.DataFrame(rows)
    y = pd.Series(ys, index=X.index, name="y")
    return X, y


def rf_predict_next_k(y_hist: pd.Series, k=24, X_cols=None, rf_pipe=None,
                      lags_short=24, weekly_lags=(168, 167, 169), roll_windows=(3, 6, 12, 24)):
    """Previsão iterativa k passos à frente com Random Forest treinada."""
    weekly_lags = tuple(L for L in weekly_lags if L < len(y_hist))
    y_work = y_hist.copy()
    preds = []
    last_ts = y_hist.index[-1]

    for i in range(1, k + 1):
        ts_i = last_ts + pd.Timedelta(hours=i)
        feats = {f"lag_{L}": y_work.iloc[-L] for L in range(1, lags_short + 1)}
        for L in weekly_lags:
            feats[f"lag_{L}"] = y_work.iloc[-L]
        for w in roll_windows:
            w_eff = min(w, len(y_work))
            feats[f"roll_mean_{w}"] = y_work.iloc[-w_eff:].mean()
            feats[f"roll_std_{w}"] = y_work.iloc[-w_eff:].std(ddof=0)

        # calendário
        feats["sin_h"] = np.sin(2 * np.pi * ts_i.hour / 24)
        feats["cos_h"] = np.cos(2 * np.pi * ts_i.hour / 24)
        feats["sin_w"] = np.sin(2 * np.pi * ts_i.dayofweek / 7)
        feats["cos_w"] = np.cos(2 * np.pi * ts_i.dayofweek / 7)

        Xi = pd.DataFrame([feats], index=[ts_i])
        if X_cols is not None:
            Xi = Xi.reindex(columns=X_cols, fill_value=0.0)

        y_hat = float(rf_pipe.predict(Xi)[0])
        preds.append(y_hat)
        y_work = pd.concat([y_work, pd.Series([y_hat], index=[ts_i])])

    return pd.Series(preds, index=pd.date_range(last_ts + pd.Timedelta(hours=1), periods=k, freq="H"))


# -------------------------------------------
# Treinar, validar, prever 24
# -------------------------------------------
def test_three_models_next24(csv_path="ws_hours.csv"):
    df = load_hourly_ws(csv_path)
    y = df["ws2m"].astype(float)

    # janela de validação (último trecho)
    val_len = max(24, min(168, int(len(y) * 0.25)))  # 25% (mín 24, máx 168)
    if val_len >= len(y):
        val_len = max(24, len(y) // 3)

    y_train = y.iloc[:-val_len]
    y_val = y.iloc[-val_len:]
    future_idx = pd.date_range(y.index[-1] + pd.Timedelta(hours=1), periods=24, freq="H")

    results = {}
    forecasts_24 = {}

    # ---------- Modelo 1: SARIMA(1,0,1)(1,1,1,24) ----------
    try:
        sarima = SARIMAX(
            y_train, order=(1, 0, 1), seasonal_order=(1, 1, 1, 24),
            enforce_stationarity=False, enforce_invertibility=False
        ).fit(disp=False)
        sarima_val = sarima.forecast(steps=val_len)
        results["SARIMA"] = rmse(y_val, sarima_val)

        sarima_full = SARIMAX(
            y, order=(1, 0, 1), seasonal_order=(1, 1, 1, 24),
            enforce_stationarity=False, enforce_invertibility=False
        ).fit(disp=False)
        forecasts_24["SARIMA"] = pd.Series(sarima_full.forecast(steps=24), index=future_idx)
    except Exception as e:
        results["SARIMA"] = np.inf
        forecasts_24["SARIMA"] = None

    # ---------- Modelo 2: Holt-Winters (add, add, 24) ----------
    try:
        hw = ExponentialSmoothing(y_train, trend="add", seasonal="add", seasonal_periods=24).fit()
        hw_val = hw.forecast(val_len)
        results["HoltWinters"] = rmse(y_val, hw_val)

        hw_full = ExponentialSmoothing(y, trend="add", seasonal="add", seasonal_periods=24).fit()
        forecasts_24["HoltWinters"] = pd.Series(hw_full.forecast(24), index=future_idx)
    except Exception as e:
        results["HoltWinters"] = np.inf
        forecasts_24["HoltWinters"] = None

    # ---------- Modelo 3: Random Forest ----------
    rf_rmse = np.inf
    rf_fc = None
    X_full, y_full = make_supervised(y)
    if X_full is not None:
        split_time = y.index[-val_len]
        X_tr = X_full.loc[X_full.index < split_time]
        y_tr = y_full.loc[X_full.index < split_time]
        X_va = X_full.loc[X_full.index >= split_time]
        y_va = y_full.loc[X_full.index >= split_time]

        if len(X_tr) > 0 and len(X_va) > 0:
            rf_pipe = Pipeline([
                ("scaler", StandardScaler(with_mean=False)),
                ("rf", RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))
            ])
            rf_pipe.fit(X_tr, y_tr)
            rf_val_pred = rf_pipe.predict(X_va)
            rf_rmse = rmse(y_va, rf_val_pred)

            # Refit full e previsão 24
            rf_pipe.fit(X_full, y_full)
            rf_fc = rf_predict_next_k(
                y_hist=y, k=24, X_cols=X_full.columns, rf_pipe=rf_pipe
            )

    results["RandomForest"] = rf_rmse
    forecasts_24["RandomForest"] = rf_fc

    # ---------- Consolidação ----------
    # imprime comparação de RMSE (validação)
    print("RMSE validação (menor é melhor):")
    for m, s in results.items():
        print(f"  {m:<13s}: {s:.4f}")

    # previsões para os próximos 24 pontos (cada modelo)
    print("\nPrevisões (próximas 24h):")
    for m, fc in forecasts_24.items():
        if fc is None:
            print(f"  {m:<13s}: [sem previsão]")
        else:
            print(f"  {m:<13s}: " + ",".join(f"{v:.2f}" for v in fc.values))

    # modelo vencedor (pelo RMSE)
    valid_items = [(m, s) for m, s in results.items() if np.isfinite(s) and forecasts_24[m] is not None]
    if valid_items:
        best_model = min(valid_items, key=lambda x: x[1])[0]
        print(f"\nMelhor modelo (validação): {best_model}")
        print("Previsão final (24h do melhor): " + ",".join(f"{v:.2f}" for v in forecasts_24[best_model].values))
    else:
        print("\nNenhum modelo pôde ser avaliado com sucesso.")


# -------------------------------------------
# Execução
# -------------------------------------------
# Ajuste o caminho/arquivo se necessário:
# Ex.: csv_path = "ws_hours.csv"
csv_path = "ws_hours.csv"
test_three_models_next24(csv_path)
